In [ ]:
from bs4 import BeautifulSoup as bs
from urllib.request import urlopen
from db_utils import connect
import pandas as pd
from mlb_player_data_utils import upload_stats, upload_directory, close_con
pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [ ]:
conn = connect('mlb_msmc')
query = "SELECT game_id,local_game_time FROM games_raw WHERE (game_type = 'R');"
games_df = pd.read_sql(query, conn)
conn.close()
games_df.sort_values(by='game_id', inplace=True)
games_df.reset_index(inplace=True)

In [ ]:
base_url = "http://gd2.mlb.com/components/game/mlb/"
stat_dfs = {'pitchers': [], 'batters': []}
players_seen, directory_dicts = [], []
game_count = 0

In [ ]:
print('{} games to be processed.\n'.format(len(games_df)))
with open('logs/error.log', 'a') as f_err:
    for game in games_df.itertuples(index=False, name='Games'):
        active_date = game[2]
        day_url = '{}year_{}/month_{:02}/day_{:02}/'.format(base_url, active_date.year,
                                                            active_date.month, active_date.day)
        game_url = day_url + 'gid_' + game[1] + '/'
        try:
            box = bs(urlopen(game_url + 'boxscore.xml'), 'lxml')
        except:
            f_err.write('Error opening {}\n'.format(game_url + 'boxscore.xml'))
            continue

        players = {}
        players['pitchers'] = [pitcher['id'] for pitcher in box.find_all('pitcher')]
        players['batters'] = [batter['id'] for batter in box.find_all('batter')]
        player_dicts = {'pitchers': [], 'batters': []}

        for p_type in ['pitchers', 'batters']:
            for player in players[p_type]:
                player_url = game_url + p_type + '/' + player + '.xml'
                try:
                    stats = bs(urlopen(player_url), 'lxml').player
                except:
                    f_err.write('Error opening {}\n'.format(player_url))
                    continue
                if stats.attrs['id'] not in players_seen:
                    player_dir = stats.attrs
                    for k in ['team', 'jersey_number']:
                        player_dir.pop(k, None)
                    player_dir['height'] = (12 * int(player_dir['height'].split('-')[0]) +
                                                 int(player_dir['height'].split('-')[1]))
                    players_seen.append(player_dir['id'])
                    directory_dicts.append(player_dir)
                stat_cats = {child.name: child.attrs for child in stats.children}
                for k in ['atbats', 'faced', 'pitch', 'vs_p', 'vs_p5', 'vs_b', 'vs_b5']:
                    stat_cats.pop(k, None)
                flattened_stats = {cat + '_' + stat: stat_cats[cat][stat]
                                   for cat in stat_cats for stat in stat_cats[cat]}
                flattened_stats['game_id'] = game[1]
                flattened_stats['player_id'] = stats.attrs['id']
                player_dicts[p_type].append(flattened_stats)
            stat_dfs[p_type].append(pd.DataFrame(player_dicts[p_type]))
                
        game_count += 1
        if game_count % 100 == 0:
            upload_stats(stat_dfs)
            stat_dfs = {'pitchers': [], 'batters': []}
            print('{} games processed and written.'.format(game_count))
                    
upload_stats(stat_dfs)
print('{} games processed and written.\n'.format(game_count))
upload_directory(directory_dicts)
print('Player directory processed and written.\nAll done.')
close_con()